In [2]:
# Import libraries/dependencies

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('darkgrid')

In [3]:
# Import the training data

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# checking the head of the dataframe

train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
# A brief summary of the data
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [4]:
# checking for missing values in the training data
#sns.heatmap(train.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [6]:
#sns.heatmap(test.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [4]:
# Creating a function that fill in null values

def fillMissing(data):
    data['Spa'] = data['Spa'].fillna(0)
    data['RoomService'] = data['RoomService'].fillna(0)
    data['VRDeck'] = data['VRDeck'].fillna(0)
    data['FoodCourt'] = data['FoodCourt'].fillna(0)
    data['ShoppingMall'] = data['ShoppingMall'].fillna(0)
    data['HomePlanet'] = data['HomePlanet'].fillna(data['HomePlanet'].mode()[0])
    data['Cabin'] = data['Cabin'].fillna(data['Cabin'].mode()[0])
    data['CryoSleep'] = data['CryoSleep'].fillna(data['CryoSleep'].mode()[0])
    data['Destination'] = data['Destination'].fillna(data['Destination'].mode()[0])
    data['VIP'] = data['VIP'].fillna(data['VIP'].mode()[0])
    data['Age'] = data['Age'].fillna(data['Age'].mean())
    data.drop(['Name'], axis=1, inplace=True)
    data.drop(['PassengerId'], axis=1, inplace=True)
    data.drop(['Cabin'], axis=1, inplace=True)

In [5]:
# Using the function to fill the missing train and test data
fillMissing(train)
fillMissing(test)

In [6]:
# Creating a function that one-hot my categorical columns
mylist = ['VIP','CryoSleep']
def OneHot(data):
    for l in mylist:
        catg = data[l].astype('category')
        c = catg.values
        data[l] = c.codes

In [ ]:
# Applying the function on my train and test data
OneHot(train)
OneHot(test)

In [7]:
# Creating one-hot for my Transported column
catg = train['Transported'].astype('category')
c = catg.values
train['Transported'] = c.codes

In [8]:
# Get dummies for the other categorical columns in the training data
home = pd.get_dummies(train['HomePlanet'], drop_first=True)
destination = pd.get_dummies(train['Destination'], drop_first=True)

train = pd.concat([train, home, destination], axis=1)
train.drop(['HomePlanet', 'Destination'], axis=1, inplace=True)

In [10]:
# Get dummies for the other categorical columns in the test data
home = pd.get_dummies(test['HomePlanet'], drop_first=True)
destination = pd.get_dummies(test['Destination'], drop_first=True)

test = pd.concat([test, home, destination], axis=1)
test.drop(['HomePlanet', 'Destination'], axis=1, inplace=True)

In [11]:
# Adding the total amount spent into a new column called Bills
train['Bills'] = train['Spa'] + train['RoomService'] + train['VRDeck'] + train['FoodCourt'] + train['ShoppingMall']
test['Bills'] = test['Spa'] + test['RoomService'] + test['VRDeck'] + test['FoodCourt'] + test['ShoppingMall']

In [12]:
# Droping the spa, roomservice, vrdeck, foodcourt and shoppingMall column
train.drop(['Spa', 'RoomService', 'VRDeck', 'FoodCourt', 'ShoppingMall'],
           axis=1,inplace=True)
test.drop(['Spa', 'RoomService', 'VRDeck', 'FoodCourt', 'ShoppingMall'],
          axis=1,inplace=True)

In [93]:
# Splitting the data into feature and label
X = train.drop(['Transported'],axis=1)  # feature
y = train['Transported']                # label

In [51]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_X = scaler.fit_(X)

In [ ]:
# Using RandomForestClassifier to extract the feature importance
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
for name, score in zip(X_train.columns, rnd_clf.feature_importances_):
    print(name, score)

In [94]:
# Splitting the training data into training and validation set
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y)

In [47]:
from sklearn.metrics import precision_score

In [48]:
# Training the model using LogisticRegression 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

log_pred = log_reg.predict(X_val)

log_acc = accuracy_score(log_pred,y_val)
log_pre = precision_score(log_pred,y_val)
print('acc :',log_acc ,'precision :',log_pre)

acc : 0.7263109475620975 precision : 0.6328545780969479


In [49]:
# Training the model using the RandomForestClassifier

from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth=8, max_features=2, n_estimators=50)
rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_val)

rfc_acc = accuracy_score(rfc_pred,y_val)
rfc_pre = precision_score(rfc_pred,y_val)
print(rfc_acc, rfc_pre)

0.7419503219871205 0.6373429084380611


In [50]:
# Training the model using an ensemble method

from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

log_reg = LogisticRegression(C=0.001)
svc_clf = SVC()
rfc_clf = RandomForestClassifier(max_depth=8, max_features=4, n_estimators=30)
#tree_clf = DecisionTreeClassifier()

voting_clf = VotingClassifier(
            estimators=[('lr',log_reg), ('sv',svc_clf),
                        ('rf',rfc_clf)],
            voting='hard')
voting_clf.fit(X_train, y_train)

for clf in (log_reg, svc_clf, rfc_clf,voting_clf):
    clf.fit(X_train,y_train)
    pred = clf.predict(X_val)
    print(clf.__class__.__name__,':',accuracy_score(pred, y_val))

LogisticRegression : 0.7387304507819688
SVC : 0.7364305427782889
RandomForestClassifier : 0.7382704691812327
VotingClassifier : 0.7387304507819688


In [90]:
# Using GridSearchCV for hyperparameter tuning
from sklearn.model_selection import GridSearchCV

log_param = {'C':[0.001,0.01,0.1,1,10,100]}

grid = GridSearchCV(log_reg, log_param, cv=5)
grid.fit(X_train, y_train)

grid.best_estimator_

LogisticRegression(C=0.001)

In [84]:
# Tuning the RandomForestClassifier
param_grid = {'n_estimators': [3, 10, 30,50,100,200], 
              'max_features': [2, 4, 6, 8],
              'max_depth':[2,4,8,16,None]}

rf_grid = GridSearchCV(rfc_clf, param_grid, cv=5)
rf_grid.fit(X_train, y_train)
rf_grid.best_estimator_

RandomForestClassifier(max_depth=8, max_features=4, n_estimators=30)

In [59]:
# Tuning the DecisionTreeClassifer
tree_param = {'max_depth':[2,3,4,5], 'max_features':[2,4,6]}

tree_grid = GridSearchCV(tree_clf, tree_param, cv=5)
tree_grid.fit(X_train, y_train)
tree_grid.best_estimator_

DecisionTreeClassifier(max_depth=5, max_features=4)

In [100]:
# Importing the test data for prediction
t = pd.read_csv('test.csv')

In [97]:
# I decided to use the RandomForestClassifer since it has the highest validation accuracy
rnf_clf  = RandomForestClassifier(max_depth=8, max_features=4, n_estimators=30)
rnf_clf.fit(X, y)
pred = rnf_clf.predict(test)

In [98]:
# Importing the sample data
sample = pd.read_csv('sample_submission.csv')

In [102]:
passengerId = t[['PassengerId']]

In [107]:
# Storing my predicted value into thisList. 1 represent True and 0 reps False
thisList = []
for val in pred:
    if val == 1:
        thisList.append(True)
    else:
        thisList.append(False)

In [111]:
transported = pd.DataFrame(thisList,columns=['Transported'])

In [113]:
submission = pd.concat([passengerId, transported],axis=1)

In [120]:
submission.to_csv('my_submission.csv',index=False)

In [121]:
sub = pd.read_csv('my_submission.csv')

In [1]:
#sub